In [2]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_json('https://s3.amazonaws.com/temp-data-pulls/newdump.json')

In [ ]:
pd.set_option('display.max_colwidth', -10)
pd.set_option('display.max_columns', None)

In [4]:
pd.options.mode.chained_assignment = None
rawpin_blog = df[(df["type"]=="pin") | (df["type"]=="blog post")]
rawpin_blog.drop(["has_spend"], axis = 1, inplace=True)
channel_info = rawpin_blog['channel_info'].apply(pd.Series)
channel_info.columns = ["channel", "info"]
content_info = rawpin_blog['content'].apply(pd.Series)
content_info.drop(['author_email', 'content', 'pinned_from'], axis=1, inplace=True) ## THESE HAVE ONLY NULLS
for x in content_info.columns:
    if "count" in x:
        content_info[x].fillna(np.NaN, inplace = True)
        #content_info[x] = content_info[x].astype(int)
master_pinblog = rawpin_blog.join(channel_info).join(content_info)
master_pinblog.drop(['channel_info', 'content'], axis = 1, inplace = True)
master_pinblog.columns = ['brand', 'engagement', 'uniqueid', 'impact', 'share_token', 'timestamp',
       'type', 'urls', 'channel', 'info', 'author_name', 'comment_count',
       'description', 'fb_likecount', 'fb_sharecount',
       'gplus_count', 'hashtags', 'image_url', 'like_count',
       'link', 'linkedin_sharecount', 'links', 'pin_id', 'pin_url',
       'pin_count', 'post_type', 'repin_count', 'summary',
       'thumbnail_url', 'title', 'tweet_count']

master_pinblog["links_count"] = master_pinblog['links'].str.len()
df_new = master_pinblog

KeyboardInterrupt: 

In [ ]:
df_new

In [ ]:
df_new.dtypes

In [ ]:
# create new df called blogs that only contains blogs
blogs = df_new[df_new.type == 'blog post']
# converts link to string so we can split
blogs.link = blogs.link.astype(str)
# instantiate a new list called new_mag
new_mag = []
# list comprehension that just keeps part before '.com'
# we can use list comprehension because this is true for all values
magazine = [i.split('.com')[0] for i in blogs.link]
# start for loop to get rid of everything before the name of the magazine
for i in magazine:
    if '.' in i:
        new_mag.append(i.split('.')[1])
# if there isn't a '.' it just sends the existing name to the list
    else:
        new_mag.append(i)
# create new column for the df with the publications
blogs['pub'] = new_mag
print(blogs.pub.unique())

In [ ]:
blogs.dtypes

In [17]:
import nltk
from stop_words import get_stop_words
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

In [ ]:
stemmer = SnowballStemmer("english",ignore_stopwords=True)
en_stop = get_stop_words('en')
# when a word is stemmed, contractions are rightfully turned into different stems since 's = is
# however, in reality, all of those words are themselves stop words, so I want to exclude them
# question marks and the like are not helpful for our purpose of figuring out potential categories
contractions = ["'s","s","'",".",",","n't","'d","ll","re","ve","``",
                "''","”","“","’","(",")","?",":","t",";","d","!","-","[","]","w","#"]
# list for tokenized documents in loop
texts = []

# loop through document list
post_text = [i for i in blogs.summary]
count = 1
print("Initializing tokenizer and stemmer...")
print("Number of posts tokenized and stemmed:")
for i in post_text:
    # clean and tokenize document string
    raw = i.lower()
    tokens = word_tokenize(raw)
    
    # stem tokens and remove stop words
    stemmed_tokens = [stemmer.stem(i) for i in tokens if not i in en_stop]
    
    #remove stemmed contractions
    contracted_tokens = [i for i in stemmed_tokens if not i in contractions]
    
    # add tokens to list
    texts.append(contracted_tokens)
    if count % 5000 == 0:
        print(count)
    count += 1
print("Stemming Completed.")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tvec20 = CountVectorizer(max_features = 20,stop_words='english')
tfidf20  = pd.DataFrame(tvec20.fit_transform(blogs.title).todense(),columns=tvec20.get_feature_names())
print("Top 20 tf-idf:")
print(tfidf20.sum().sort_values(ascending=False))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tvec40 = CountVectorizer(max_features = 40,stop_words='english')
tfidf40  = pd.DataFrame(tvec40.fit_transform(blogs.title).todense(),columns=tvec40.get_feature_names())
print("Top 40 tf-idf:")
print(tfidf40.sum().sort_values(ascending=False))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cvec40 = CountVectorizer(max_features = 40,stop_words='english')
vec40  = pd.DataFrame(cvec40.fit_transform(blogs.summary).todense(),columns=cvec40.get_feature_names())
print("Top 40 Most Used Words:")
print(vec40.sum().sort_values(ascending=False))

In [ ]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

In [ ]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1)
print(ldamodel.print_topics(num_topics=10, num_words=10))

In [ ]:
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [3]:
blogposts = pd.read_csv('/Users/Scott/Documents/DSI 6 /Repo /TrackMaven/notebooks/BlogMaster.csv')

/Users/Scott/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#Changing the type of the coloumn to string format 
blogposts.blogpost = blogposts.blogpost.astype(str)

In [13]:
# Putting the blogs into a list format 
string = [i for i in blogposts.blogpost]

In [14]:
import re 

In [15]:
result = [re.sub(r"http\S+", "", i) for i in string]

In [16]:
result = [re.sub("\S{13,}", "", i) for i in string]
''' Here I am getting rid of the URLs. The URLs are annoyingly connected to other words so I decided to just 
get rid of words that are over 13 characters long.'''

' Here I am getting rid of the URLs. The URLs are annoyingly connected to other words so I decided to just \nget rid of words that are over 13 characters long.'

In [18]:
blogposts.blogpost = result
# Reconnecting the code back to the dataframe 

In [19]:
from textblob import TextBlob
#importing an NLP python language to find sentiment for the text 

In [21]:
wiki = [TextBlob(i).sentiment for i in blogposts.blogpost]
#Itterating through the blogposts and assigning the TextBlob attribute to them.

In [22]:
TextBlob('Dogs are the best kind pets').sentiment
#How this code is working. Poliarity is how positive or negative the text is. 1 being positive and -1 being negative
#Subjectivity is how subjective the text is. 1 Being very subjective and 0 being objective. 

Sentiment(polarity=0.8, subjectivity=0.6)

In [23]:
blogposts['blog_polarity'] = [wiki[i].polarity for i in range(len(wiki))]
# I am iterating through Blogs and assining a polarity to them. Then I am putting them into a new column 

In [24]:
blogposts['blog_subjectivity'] = [wiki[i].subjectivity for i in range(len(wiki))]
# I am iterating through Blogs and assining a subjectivity to them. Then I am putting them into a new column 

In [25]:
blogposts.head()

,Unnamed: 0,brand,engagement,uniqueid,impact,share_token,timestamp,type,urls,channel,...,summary,thumbnail_url,title,tweet_count,links_count,pub,blogpost,blogdetails,polarity,subjectivity
0,0,137316,33,MTM3MzE2LTIwMTQ2MTUyX2Jsb2cgcG9zdF8xNzQ4Mw,0.057249,MUEF3US5L1fQ9o23xiryJN3K85bXD5n2I0Md_RII3LY,2017-11-16 02:00:00,blog post,[{'url': 'https://www.glamour.com/story/riverd...,blog,...,This week's episode of 'Riverdale' unveiled th...,https://media.glamour.com/photos/5a0cb482164c4...,'Riverdale' Season 2 Episode 6 Recap: Who Is t...,0.0,0,glamour,Tonight's episode of was pretty standard: Jug...,"{'@type': 'NewsArticle', 'headline': ""The Blac...",-0.073446,0.480521
1,1,137322,1,MTM3MzIyLTIwMTQ0MDE0X2Jsb2cgcG9zdF8xNzQ4Mw,0.000927,B0yFf6kIDBS9-poQ1Fyv2t3K85bXD5n2I0Md_RII3LY,2017-11-16 02:00:00,blog post,[{'url': 'https://www.teenvogue.com/story/rive...,blog,...,"Find out what happened on ""Riverdale"" season 2...",https://assets.teenvogue.com/photos/5a0b864107...,"""Riverdale"" Recap Season 2 Episode 6: Who Is t...",0.0,0,teenvogue,"__Each week, *Teen Vogue* will be taking you o...","{'@type': 'NewsArticle', 'headline': '""Riverda...",0.079418,0.456985
2,2,137329,1077,MTM3MzI5LTIwMTQ0NzQ2X2Jsb2cgcG9zdF8xNzQ4Mw,2.625943,jByu9k_eNjDs0NyPjDTqvt3K85bXD5n2I0Md_RII3LY,2017-11-16 01:05:28,blog post,[{'url': 'https://www.wmagazine.com/story/leon...,blog,...,"Leonardo da Vinci's once-lost ""Salvator Mundi""...",https://media.wmagazine.com/photos/59dd2ae5a46...,Leonardo da Vinci's Salvator Mundi Painting Se...,0.0,0,wmagazine,"Last month, it was revealed that a once-lost [...","{'@type': 'NewsArticle', 'headline': ""Leonardo...",0.009103,0.542439
3,3,137322,1,MTM3MzIyLTIwMTQ0MDE1X2Jsb2cgcG9zdF8xNzQ4Mw,0.000937,2l5jpXZ34bmotrXo0V_z1t3K85bXD5n2I0Md_RII3LY,2017-11-15 23:44:52,blog post,[{'url': 'https://www.teenvogue.com/story/made...,blog,...,"""F the Prom,"" a new movie starring Danielle Ca...",https://assets.teenvogue.com/photos/5a0cc15c11...,Madelaine Petsch and Danielle Campbell Star in...,0.0,0,teenvogue,[Madelaine has proved on that she can play a...,"{'@type': 'NewsArticle', 'headline': '""Mean Gi...",0.248575,0.427420
4,4,137329,3,MTM3MzI5LTIwMTQ0NzQ3X2Jsb2cgcG9zdF8xNzQ4Mw,0.006514,HUE4eXJv3ImpRfBokd2d5N3K85bXD5n2I0Md_RII3LY,2017-11-15 23:39:41,blog post,[{'url': 'https://www.wmagazine.com/story/get-...,blog,...,While Get Out has been submitted to the Golden...,https://media.wmagazine.com/photos/58ac4733aaa...,Is Get Out a Comedy or a Drama? Jordan Peele W...,0.0,0,wmagazine,Filmmaker [Jordan has made a name for himself...,"{'@type': 'NewsArticle', 'headline': 'Is Get O...",0.160922,0.501335


In [26]:
string1 = [i for i in blogposts.title]

In [84]:
string1[4]

'Is Get Out a Comedy or a Drama? Jordan Peele Weighs in on the Debate'

In [27]:
result1 = [re.sub(r"http\S+", "", i) for i in string1]

In [28]:
result1 = [re.sub("\S{13,}", "", i) for i in string1]

In [29]:
blogposts.title = result1

In [40]:
wiki1 = [TextBlob(i).sentiment for i in blogposts.title]

In [77]:
scott = TextBlob("People Sexiest Man Alive Blake Shelton's Homophobic and Racist Tweets")
scott.sentiment

Sentiment(polarity=0.1, subjectivity=0.4)

In [85]:
scott = TextBlob('Is Get Out a Comedy or a Drama? Jordan Peele Weighs in on the Debate')
scott.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [31]:
blogposts['title_polarity'] = [wiki1[i].polarity for i in range(len(wiki1))]

In [32]:
blogposts['title_subjectivity'] = [wiki1[i].subjectivity for i in range(len(wiki1))]

In [33]:
blogposts.head()

,Unnamed: 0,brand,engagement,uniqueid,impact,share_token,timestamp,type,urls,channel,...,title,tweet_count,links_count,pub,blogpost,blogdetails,polarity,subjectivity,title_polarity,title_subjectivity
0,0,137316,33,MTM3MzE2LTIwMTQ2MTUyX2Jsb2cgcG9zdF8xNzQ4Mw,0.057249,MUEF3US5L1fQ9o23xiryJN3K85bXD5n2I0Md_RII3LY,2017-11-16 02:00:00,blog post,[{'url': 'https://www.glamour.com/story/riverd...,blog,...,'Riverdale' Season 2 Episode 6 Recap: Who Is t...,0.0,0,glamour,Tonight's episode of was pretty standard: Jug...,"{'@type': 'NewsArticle', 'headline': ""The Blac...",-0.073446,0.480521,0.0,0.0
1,1,137322,1,MTM3MzIyLTIwMTQ0MDE0X2Jsb2cgcG9zdF8xNzQ4Mw,0.000927,B0yFf6kIDBS9-poQ1Fyv2t3K85bXD5n2I0Md_RII3LY,2017-11-16 02:00:00,blog post,[{'url': 'https://www.teenvogue.com/story/rive...,blog,...,"""Riverdale"" Recap Season 2 Episode 6: Who Is t...",0.0,0,teenvogue,"__Each week, *Teen Vogue* will be taking you o...","{'@type': 'NewsArticle', 'headline': '""Riverda...",0.079418,0.456985,0.0,0.0
2,2,137329,1077,MTM3MzI5LTIwMTQ0NzQ2X2Jsb2cgcG9zdF8xNzQ4Mw,2.625943,jByu9k_eNjDs0NyPjDTqvt3K85bXD5n2I0Md_RII3LY,2017-11-16 01:05:28,blog post,[{'url': 'https://www.wmagazine.com/story/leon...,blog,...,Leonardo da Vinci's Salvator Mundi Painting Se...,0.0,0,wmagazine,"Last month, it was revealed that a once-lost [...","{'@type': 'NewsArticle', 'headline': ""Leonardo...",0.009103,0.542439,0.0,0.0
3,3,137322,1,MTM3MzIyLTIwMTQ0MDE1X2Jsb2cgcG9zdF8xNzQ4Mw,0.000937,2l5jpXZ34bmotrXo0V_z1t3K85bXD5n2I0Md_RII3LY,2017-11-15 23:44:52,blog post,[{'url': 'https://www.teenvogue.com/story/made...,blog,...,Madelaine Petsch and Danielle Campbell Star in...,0.0,0,teenvogue,[Madelaine has proved on that she can play a...,"{'@type': 'NewsArticle', 'headline': '""Mean Gi...",0.248575,0.427420,0.0,0.0
4,4,137329,3,MTM3MzI5LTIwMTQ0NzQ3X2Jsb2cgcG9zdF8xNzQ4Mw,0.006514,HUE4eXJv3ImpRfBokd2d5N3K85bXD5n2I0Md_RII3LY,2017-11-15 23:39:41,blog post,[{'url': 'https://www.wmagazine.com/story/get-...,blog,...,Is Get Out a Comedy or a Drama? Jordan Peele W...,0.0,0,wmagazine,Filmmaker [Jordan has made a name for himself...,"{'@type': 'NewsArticle', 'headline': 'Is Get O...",0.160922,0.501335,0.0,0.0
